# YeastMate detection with the Python API

The following code is a short example of how to use the Python API to detect yeast on 2D sample images.

In [ ]:
# You can use glob to find all files in a directory

from glob import glob

imagelist = glob('./images/*.tif')

In [ ]:
# YeastMate expects 2D Numpy-arrays as input

from skimage.io import imread

testimage = imread(imagelist[0])

print(testimage.shape)

Inference is done with the YeastMatePredictor class in inference.py. It needs to be initialized with a configuration and a model weight file.

It comes with with a convencience function inference, which expects a 2D image and returns a dictionary of detected objects and a segmentation mask.

In [ ]:
from yeastmatedetector.inference import YeastMatePredictor

# You need to copy the model to your local machine for the code to successfully run
predictor = YeastMatePredictor('../models/yeastmate.yaml', '../models/yeastmate_weights.pth')

In [ ]:
detections, mask = predictor.inference(testimage)

The segmentation mask is a 2D array of integers, where each cell is assigned a different value.

In [ ]:
from matplotlib import pyplot as pyplot

fig, ax = plt.subplots(1,2)

ax[0,0].imshow(testimage)
ax[0,1].imshow(mask)

The detected objects come in a dictionary, where each key is the unique identifier for the object. In the case of single cells and mother/daughter cells this identifier corresponds to its value in the segmentation mask. 

Each object comes with a dictionary of properties, which can be accessed by the key. It contains its id, its bounding box, a list of classes and a list of probabilities scores. An object can have multiple classes, as each cell is counted as a single cell as well as a potential member of a compound class.

In [ ]:
print(list(detections.values())[0])

In [ ]:
# You can easily save the mask as an image and the detections as a json file

import json
from skimage.io import imsave

imsave(imagelist[0].replace('.tif', '_mask.tif'), mask)

with open(imagelist[0].replace('.tif', '_detections.json'), 'w') as file:
    json.dump(detections, file, indent=1)

In [ ]:
# If you want to save the detected objects in the YeastMate format, you need to add some additional metadata

resdict = {'image': os.path.basename(imagelist[0]), 'metadata': {}, 'detections': detections}

# If your whole image is not 2D, you can set the position of your 2D slice for the detection here
framedict = {'t':"", "z":"", "c":""}
                
resdict['metadata']['height'] = testimage.shape[0]
resdict['metadata']['width'] = testimage.shape[1]
resdict['metadata']['detection_frame'] = framedict
resdict['metadata']['source'] = 'Detection'
resdict['metadata']['bbox_format'] = 'x1y1x2y2'

with open(imagelist[0].replace('.tif', '_detections.json'), 'w') as file:
    json.dump(resdict, file, indent=1)

The main inference function comes with multiple parameters, which can be used to adjust the inference process.

* score_thresholds: default {0:0.9, 1:0.75, 2:0.75}; a dictionary of score thresholds for each class. Class 0 are the single cells in the segmentation mask, class 1 are mating events and class 2 are budding evenets. You can adjust these values to your needs, if you need to reduce false negatives (lower the thresholds) or reduce false positives (increase the thresholds).

* pixel_size: default 110; if your pixel size differs from the default, you can adjust this value and your images will be automatically resized to match the scale of images YeastMate was trained on.

* reference_pixel_size: default 110; sets the reference pixel size that the parameter above is compared against. You only need to adjust this value if you re-trained the model on your own data.

* lower_quantile: default 1.5; sets the lower percentile for normalization of the images. This is used to normalize the images to a range of 0-1. The default value returns good results for most images.

* upper_quantile: default 98.5; sets the upper percentile for normalization of the images. This is used to normalize the images to a range of 0-1. The default value returns good results for most images.

If you want to normalize and rescale your images yourself, you can either set the quantiles to 0 and 100, or you can also skip the inference function and access the model directly.

The model expects a dictionary of the following format:

image = {"image": torch.Tensor[3, h, w], "height": int, "width": int}

You can also use the static YeastMatePredictor.image_to_tensor function to convert your images to the correct format. This function expects a 2D Numpy array with values ranging from 0-1.

In [ ]:
print(testimage.shape)

imagedict = YeastMatePredictor.image_to_tensor(testimage)

print(imagedict)

instances = predictor.model([imagedict])[0]['instances']

This will return the raw instances, which we need to postprocess to get the segmentation mask and the detected objects.
